In [3]:
import os, sys
currentFolder = os.path.abspath('')
try:
    sys.path.remove(str(currentFolder))
except ValueError: # Already removed
    pass

projectFolder = 'C:/Users/abjawad/Documents/GitHub/local-attention-model'
sys.path.append(str(projectFolder))
os.chdir(projectFolder)
print( f"current working dir{os.getcwd()}")


current working dirC:\Users\abjawad\Documents\GitHub\local-attention-model


In [4]:

import torch
from  torch.utils.data import DataLoader
import torch.nn as nn
from models.builder import EncoderDecoder as segmodel
from dataloader.cfg_defaults import get_cfg_defaults
from utils.lr_policy import WarmUpPolyLR
from utils.init_func import init_weight, group_weight
from config_cityscapes import *
import yaml
import os
from dataloader.cityscapes_dataloader import CityscapesDataset

c:\Users\abjawad\.conda\envs\pt_cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
config_path = 'C:/Users/abjawad/Documents/GitHub/local-attention-model/dataloader/cityscapes_rgbd_config.yaml'
# with open(config_path) as info:
#     info_dict = yaml.load(info, Loader=yaml.FullLoader)
cfg = get_cfg_defaults()
cfg.merge_from_file(config_path)
cfg.freeze()

# torch.cuda.set_per_process_memory_fraction(0.3, device=0)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

In [ ]:
# for k, v in cfg.items():
#     print(f"{k}:{v}")

In [6]:
cityscapes_train = CityscapesDataset(cfg, split='train')
train_loader = DataLoader(cityscapes_train, batch_size=4, shuffle=True, num_workers=4, drop_last=True)
print(f'total train: {len(cityscapes_train)} t_iteration:{len(train_loader)}')
cityscapes_val = CityscapesDataset(cfg, split='val')
val_loader = DataLoader(cityscapes_val, batch_size=1, shuffle=False, num_workers=4)
print(f'total val: {len(cityscapes_val)} v_iteration:{len(val_loader)}')

Using RGB input
Using RGB input
Found 2975 train images
total train: 2975 t_iteration:743
Using RGB input
Using RGB input
Found 500 val images
total val: 500 v_iteration:500


In [7]:
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=config.background)
BatchNorm2d = nn.BatchNorm2d


In [8]:
model=segmodel(cfg=config, criterion=criterion, norm_layer=BatchNorm2d)


08 20:51:46 Using backbone: Segformer-B2
08 20:51:48 Using MLP Decoder
08 20:51:48 Loading pretrained model: C:\Users\abjawad\Documents\GitHub\local-attention-model\pretrained/mit_b2.pth
08 20:51:48 Load model, Time usage:
	IO: 0.04294443130493164, initialize parameters: 0.026922225952148438
08 20:51:48 Initing weights ...


In [9]:
params_list = []
params_list = group_weight(params_list, model, BatchNorm2d, config.lr)        
optimizer = torch.optim.AdamW(params_list, lr=config.lr, betas=(0.9, 0.999), weight_decay=config.weight_decay)

In [10]:
total_iteration = config.nepochs * config.niters_per_epoch
lr_policy = WarmUpPolyLR(config.lr, config.lr_power, total_iteration, config.niters_per_epoch * config.warm_up_epoch)
print(f'lr_policy:{vars(lr_policy)}')

lr_policy:{'start_lr': 6e-05, 'lr_power': 1, 'total_iters': 186000.0, 'warmup_steps': 1488}


In [11]:
model = nn.DataParallel(model, device_ids = config.device_ids)
model.to(f'cuda:{model.device_ids[0]}', non_blocking=True)


DataParallel(
  (module): EncoderDecoder(
    (backbone): mit_b2(
      (patch_embed1): OverlapPatchEmbed(
        (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed2): OverlapPatchEmbed(
        (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed3): OverlapPatchEmbed(
        (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed4): OverlapPatchEmbed(
        (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (block1): ModuleList(
        (0): Block(
          (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
       

In [12]:
for epoch in range(1, 2):
    model.train()
    optimizer.zero_grad()
    sum_loss = 0
    m_iou_batches = 0
    for idx, sample in enumerate(train_loader):
        imgs = sample['image']
        gts = sample['label']
        imgs = imgs.to(f'cuda:{model.device_ids[0]}', non_blocking=True)
        gts = gts.to(f'cuda:{model.device_ids[0]}', non_blocking=True)  
        loss, out = model(imgs, gts)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        # sum_loss += loss
        print(f'epoch:{epoch} iteration:{idx} loss:{loss}')
    # print(f'epoch:{epoch}')

q:  torch.Size([4, 1, 65536, 64])
k:  torch.Size([4, 1, 1024, 64])
v:  torch.Size([4, 1, 1024, 64])
q:  torch.Size([4, 1, 128, 128, 4, 64])
k:  torch.Size([4, 1, 128, 128, 4, 64])
v:  torch.Size([4, 1, 128, 128, 4, 64])
q:  torch.Size([4, 1, 64, 64, 16, 64])
k:  torch.Size([4, 1, 64, 64, 16, 64])
v:  torch.Size([4, 1, 64, 64, 16, 64])
q:  torch.Size([4, 1, 32, 32, 64, 64])
k:  torch.Size([4, 1, 32, 32, 64, 64])
v:  torch.Size([4, 1, 32, 32, 64, 64])
q:  torch.Size([4, 1, 65536, 64])
k:  torch.Size([4, 1, 1024, 64])
v:  torch.Size([4, 1, 1024, 64])
q:  torch.Size([4, 1, 128, 128, 4, 64])
k:  torch.Size([4, 1, 128, 128, 4, 64])
v:  torch.Size([4, 1, 128, 128, 4, 64])
q:  torch.Size([4, 1, 64, 64, 16, 64])
k:  torch.Size([4, 1, 64, 64, 16, 64])
v:  torch.Size([4, 1, 64, 64, 16, 64])
q:  torch.Size([4, 1, 32, 32, 64, 64])
k:  torch.Size([4, 1, 32, 32, 64, 64])
v:  torch.Size([4, 1, 32, 32, 64, 64])
q:  torch.Size([4, 1, 65536, 64])
k:  torch.Size([4, 1, 1024, 64])
v:  torch.Size([4, 1, 102

KeyboardInterrupt: 

In [ ]:
from torchview import draw_graph

input_size = (2, 19200, 32)
# device='meta' -> no memory is consumed for visualization
model_graph = draw_graph(model, input_size=input_size, device='meta')
model_graph.visual_graph

In [ ]:
from torchsummary import summary

# Assuming you have already created and loaded your 'model'
model = segmodel(cfg=config, criterion=criterion, norm_layer=BatchNorm2d)
# Load the pretrained weights (if not already loaded)
model.load_state_dict(torch.load('C:/Users/abjawad/Documents/GitHub/local-attention-model/pretrained/mit_b2.pth'))

# Specify the input shape (e.g., (channels, height, width))
input_shape = (4, 19200, 32)  # Adjust the input shape according to your data

# Use torchsummary to display the model summary
summary(model, input_size=input_shape, device='cpu')  # 'cpu' or 'cuda' depending on your device
